In [1]:
import pandas as pd
import psycopg2
import sys
import xarray as xr

from getpass import getpass
from subprocess import call, check_output

List of files to process

In [2]:
path = "/data-uerra/mescan-surfex/precipitation/"

In [3]:
len(path)

40

In [4]:
!ls {path}

consecutive_dry_days_index_per_time_period.tiff
consecutive_wet_days_index_per_time_period.tiff
heavy_precipitation_days_index_per_time_period.tiff
highest_five_day_precipitation_amount_per_time_period.tiff
highest_one_day_precipitation_amount_per_time_period.tiff
mescan_surfex_nordics_consecutive_dry_days_index_per_time_period.sql
mescan_surfex_nordics_consecutive_wet_days_index_per_time_period.sql
mescan-surfex-nordics-eca_cdd.nc
mescan-surfex-nordics-eca_cwd.nc
mescan-surfex-nordics-eca_pd.nc
mescan-surfex-nordics-eca_r10mm.nc
mescan-surfex-nordics-eca_r20mm.nc
mescan-surfex-nordics-eca_rr1.nc
mescan-surfex-nordics-eca_rx1day.nc
mescan-surfex-nordics-eca_rx5day.nc
mescan-surfex-nordics-eca_sdii.nc
mescan_surfex_nordics_heavy_precipitation_days_index_per_time_period.sql
mescan_surfex_nordics_highest_five_day_precipitation_amount_per_time_period.sql
mescan_surfex_nordics_highest_one_day_precipitation_amount_per_time_period.sql
mescan_surfex_nordics_number_of_5day_heavy_precipitation_p

In [5]:
files = [
    "mescan-surfex-nordics-precipitation_ymon.nc",
    "mescan-surfex-nordics-eca_cdd.nc",
    "mescan-surfex-nordics-eca_cwd.nc",
    "mescan-surfex-nordics-eca_pd.nc",
    "mescan-surfex-nordics-eca_r10mm.nc",
    "mescan-surfex-nordics-eca_r20mm.nc",
    "mescan-surfex-nordics-eca_rr1.nc",
    "mescan-surfex-nordics-eca_rx1day.nc",
    "mescan-surfex-nordics-eca_rx5day.nc",
    "mescan-surfex-nordics-eca_sdii.nc"
]

Build list of variables for each file

In [6]:
variables = {}
for f in files:
    ds = xr.open_dataset(path + f)
    for v in ds.variables:
        if f not in variables:
            variables[f] = []
        if v not in ["time", "lon", "lat", "inate.", "oordinate."]:
            variables[f].append(v)

print(variables)

{'mescan-surfex-nordics-precipitation_ymon.nc': ['tp'], 'mescan-surfex-nordics-eca_cdd.nc': ['consecutive_dry_days_index_per_time_period', 'number_of_cdd_periods_with_more_than_5days_per_time_period'], 'mescan-surfex-nordics-eca_cwd.nc': ['consecutive_wet_days_index_per_time_period', 'number_of_cwd_periods_with_more_than_5days_per_time_period'], 'mescan-surfex-nordics-eca_pd.nc': ['precipitation_days_index_per_time_period'], 'mescan-surfex-nordics-eca_r10mm.nc': ['heavy_precipitation_days_index_per_time_period'], 'mescan-surfex-nordics-eca_r20mm.nc': ['very_heavy_precipitation_days_index_per_time_period'], 'mescan-surfex-nordics-eca_rr1.nc': ['wet_days_index_per_time_period'], 'mescan-surfex-nordics-eca_rx1day.nc': ['highest_one_day_precipitation_amount_per_time_period'], 'mescan-surfex-nordics-eca_rx5day.nc': ['highest_five_day_precipitation_amount_per_time_period', 'number_of_5day_heavy_precipitation_periods_per_time_period'], 'mescan-surfex-nordics-eca_sdii.nc': ['simple_daily_inten

Convert each variable to a single GeoTiff file

In [7]:
for f in files:
    print(f)
    for v in variables[f]:
        g = "%s.tiff" % v
        print(g) 
        call(["gdalwarp", "-s_srs", "EPSG:4326", "-t_srs", "EPSG:3035", "NETCDF:%s/%s:%s" % (path, f, v), 
              "-of", "GTiff", "%s/%s" % (path, g)])

mescan-surfex-nordics-precipitation_ymon.nc
tp.tiff
mescan-surfex-nordics-eca_cdd.nc
consecutive_dry_days_index_per_time_period.tiff
number_of_cdd_periods_with_more_than_5days_per_time_period.tiff
mescan-surfex-nordics-eca_cwd.nc
consecutive_wet_days_index_per_time_period.tiff
number_of_cwd_periods_with_more_than_5days_per_time_period.tiff
mescan-surfex-nordics-eca_pd.nc
precipitation_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_r10mm.nc
heavy_precipitation_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_r20mm.nc
very_heavy_precipitation_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_rr1.nc
wet_days_index_per_time_period.tiff
mescan-surfex-nordics-eca_rx1day.nc
highest_one_day_precipitation_amount_per_time_period.tiff
mescan-surfex-nordics-eca_rx5day.nc
highest_five_day_precipitation_amount_per_time_period.tiff
number_of_5day_heavy_precipitation_periods_per_time_period.tiff
mescan-surfex-nordics-eca_sdii.nc
simple_daily_intensity_index_per_time_period.

In [8]:
password = getpass()

········


In [22]:
conn = psycopg2.connect("dbname='ccgeodb' user='linda_ml' host='10.0.18.10' password='%s'" % (password))

Generate SQL script to load each GeoTiff, create tables in the database and store the SQL files

In [23]:
cursor = conn.cursor()
for f in files:
    for v in variables[f]:
        g = "%s.tiff" % v
        table = "mescan_surfex_nordics_%s" % g[0:-5]
        print(g)
        print(table)
        sql = f'DROP TABLE IF EXISTS "climate"."{table}"'
        print(sql)
        cursor.execute(sql)
        
        output = check_output(["raster2pgsql", "-I", "%s%s" % (path, g), "climate.%s" % table]).decode(sys.stdout.encoding)
        statements = output.split('\n')
    
        # skip BEGIN and COMMIT statements
        for i in range(1, 4):
            if i == 1:
                print(statements[i])
            
            cursor.execute(statements[i])
        conn.commit()
        
        # Save to SQL file
        with open('%s%s.sql' % (path, table), 'w') as sql:
            sql.write(output)

tp.tiff
mescan_surfex_nordics_tp
DROP TABLE IF EXISTS "climate"."mescan_surfex_nordics_tp"
CREATE TABLE "climate"."mescan_surfex_nordics_tp" ("rid" serial PRIMARY KEY,"rast" raster);
consecutive_dry_days_index_per_time_period.tiff
mescan_surfex_nordics_consecutive_dry_days_index_per_time_period
DROP TABLE IF EXISTS "climate"."mescan_surfex_nordics_consecutive_dry_days_index_per_time_period"
CREATE TABLE "climate"."mescan_surfex_nordics_consecutive_dry_days_index_per_time_period" ("rid" serial PRIMARY KEY,"rast" raster);
number_of_cdd_periods_with_more_than_5days_per_time_period.tiff
mescan_surfex_nordics_number_of_cdd_periods_with_more_than_5days_per_time_period
DROP TABLE IF EXISTS "climate"."mescan_surfex_nordics_number_of_cdd_periods_with_more_than_5days_per_time_period"
CREATE TABLE "climate"."mescan_surfex_nordics_number_of_cdd_periods_with_more_than_5days_per_time_period" ("rid" serial PRIMARY KEY,"rast" raster);
consecutive_wet_days_index_per_time_period.tiff
mescan_surfex_nordi

In [1]:
!ls /data-uerra/mescan-surfex/precipitation/*.sql

/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_consecutive_dry_days_index_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_consecutive_wet_days_index_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_heavy_precipitation_days_index_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_highest_five_day_precipitation_amount_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_highest_one_day_precipitation_amount_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_number_of_5day_heavy_precipitation_periods_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_number_of_cdd_periods_with_more_than_5days_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_nordics_number_of_cwd_periods_with_more_than_5days_per_time_period.sql
/data-uerra/mescan-surfex/precipitation/mescan_surfex_n